In [1]:
import pandas as pd
import numpy as np
from utils import zebra
import re
solver_data = pd.read_excel("2020_Partner_Solver_original.xlsx", sheet_name='Solver Data')
partner_data =  pd.read_excel("2020_Partner_Solver_original.xlsx", sheet_name='Partner Data')
solver_data = solver_data.rename(columns={'Solution Name': 'Org'})
partner_data = partner_data.rename(columns={'Organization Name': 'Org',
                                            'Challenge':'Challenge Preference',
                                            'Stage': 'Solution Preference: Organization Stage',
                                            'Expertise': 'Partnership Preference: Non-Financial',
                                           'Geography': 'Geo Interests'})

partner_data = partner_data.replace(np.nan, "Noval")
solver_data  = solver_data.replace(np.nan, "Noval")
solver_data_copy = solver_data.copy()

In [2]:

def split_collect(df_cols, delimiter=','):
    """
        Split each value in a cell based on a delimiter 
        and return a list of unique options 
        
    """
    opts = df_cols.apply(lambda x :  x.split(delimiter)   ).to_list()
    flatten_opts = [x.strip() for y in opts for x in y ]
    opts = pd.DataFrame(data=flatten_opts, columns=['options'])
    opts = opts['options'].value_counts().index.to_list()
    return opts


def expand_col(df_col, delimiter=',',col_name='new_col'): 
    """
    Take in a pandas series whose elements are
    a string. Split each cell of the series with
    a delimiter which is used togenerate an N column dataframe. 
    N is the longest list amongst the cells of df_col after
    they have been split
    
    """
#     df_col = df_col.apply(lambda x : x.str.split(delimiter)).to_list()
    df_col = df_col.str.split(delimiter).to_list()
    new_df = pd.DataFrame(data=df_col)
    ncols = len(new_df.columns)
    new_names = []
    for x in range(1, ncols+1): 
        new_name ="".join((col_name,'_', str(x)))
        new_df = new_df.rename(columns={x-1:new_name})
    return new_df


def match_single_to_multi(single_df, multi_df, single_match_on='None'): 
    """
    Generate a pivot table between a df which has a single of choices 
    and a df with multiple columns of choices
    
    """
    melted_df = pd.melt(multi_df,id_vars='Org')
    melted_df = melted_df.drop(columns='variable')
    melted_df = melted_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    single_df = single_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    matched_df = pd.merge(melted_df, single_df, how='outer', left_on='value', right_on=single_match_on)
    matched_df = matched_df.replace(np.nan, 'Noval')
    matched_df['value'] = matched_df['value'].apply(lambda x : 0 if x == None else 1)
    pivot_table = pd.pivot_table(matched_df, index='Org_x', columns=['Org_y'], values='value', dropna=False,  aggfunc=np.sum)

    return pivot_table


def match_multi(df1, df2):
    """
    Match a feature with multiple options to another option with multiple options
    """
    
    melted_df1 = pd.melt(df1,id_vars='Org').fillna('Noval')
    melted_df2 = pd.melt(df2, id_vars='Org').fillna('Noval')
    melted_df1 = melted_df1.drop(columns='variable')
    melted_df2 = melted_df2.drop(columns='variable')
    melted_df1 = melted_df1.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    melted_df2 = melted_df2.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    matched_df = pd.merge(melted_df1, melted_df2, how='outer', left_on='value', right_on='value')
    matched_df = matched_df.replace(np.nan, 'Noval')
    matched_df['value'] = matched_df['value'].apply(lambda x : 0 if  x == 'Noval' else 1)
    pivot_table = pd.pivot_table(matched_df, index='Org_x', columns=['Org_y'], values='value',aggfunc=np.sum)
    return pivot_table


## Challenge column cleaning and matching


In [3]:
partner_challenge  = partner_data['Challenge Preference']
partner_data['Challenge Preference'] = partner_data['Challenge Preference'].apply(lambda x: x.strip().replace(';',',') if ';' in x else x.strip() )
partner_challenge_cols = expand_col(partner_data['Challenge Preference'], col_name ='Challenge')
partner_challenge_opts = split_collect(partner_data['Challenge Preference'])


In [4]:
solver_challenge_opts = split_collect(solver_data['Challenge'])


In [5]:
set(solver_challenge_opts).issubset(set(partner_challenge_opts))

True

The above shows that all options in solvers are a subset of patners so no work required in correction! Next up is getting the matching sheet


## Add challenge columns to partner data sheet

In [6]:
partner_data_updated = partner_data.copy()
partner_data_updated = partner_data_updated.drop(columns='Challenge Preference')
partner_data_updated = pd.concat([partner_data_updated, partner_challenge_cols], axis=1)



## Geo Interests

In [7]:
partner_data_updated['Geo Interest'] = partner_data_updated['Geo Interest'].apply(lambda x: x.strip().replace(';',',') if ';' in x else x.strip() )
partner_geo_opts = split_collect(partner_data_updated['Geo Interest'])
partner_geo_opts

['North America',
 'East Asia and Pacific',
 'Sub-Saharan Africa',
 'Europe and Central Asia',
 'Latin America and the Caribbean',
 'Middle East and North Africa',
 'South Asia',
 'Oceania',
 'Noval']

In [8]:
solver_geo_opts = solver_data[['Geo 1', 'Geo 2', 'Geo 3']].apply(lambda x : ",".join(x)   ,axis=1)
solver_geo_opts = split_collect(solver_geo_opts)
solver_geo_opts

['Noval',
 'North America',
 'Sub-Saharan Africa',
 'South Asia',
 'Middle East and North Africa',
 'Latin America and the Caribbean',
 'Europe and Central Asia',
 'East Asia and Pacific']

In [9]:
set(solver_geo_opts).issubset(set(partner_geo_opts))

True

### Add geo columns to partner datasheet

In [10]:
partner_geo_cols = partner_data_updated['Geo Interest']

partner_geo_cols = expand_col(partner_geo_cols, col_name='geo')

# remove partner geo column and add individual geo columns
partner_data_updated = pd.concat([partner_data_updated, partner_geo_cols], axis=1)
partner_data_updated = partner_data_updated.drop(columns=['Geo Interest'])


## Key needs

In [11]:
# Partnership preferences non financal from partner data

pref_name = 'Partnership Preference'
prefs_columns = [x  for x in partner_data_updated.columns if pref_name in x]
partner_prefs_opts = partner_data_updated[prefs_columns]
partner_prefs_opts = split_collect(partner_prefs_opts.apply(lambda x : ";".join(x)   ,axis=1), delimiter=';')
partner_prefs_opts

['Noval',
 'Business model (e.g. product-market fit, strategy & development)',
 'Product / Service Distribution (e.g. expanding client base)',
 'Monitoring & Evaluation (e.g. collecting/using data, measuring impact)',
 'Human Capital (e.g. sourcing talent, board development, etc.)',
 'Financial (e.g. improving accounting practices, pitching to investors)',
 'Technology (e.g. software or hardware, web development/design, data analysis, etc.)',
 'Public Relations (e.g. branding/marketing strategy, social and global media)',
 'Other (explain below)',
 'Legal or Regulatory Matters']

In [12]:
needs_name = 'Key Need'
needs_columns = [x  for x in solver_data.columns if needs_name in x]
solver_needs_opts = solver_data[needs_columns]
solver_needs_opts = split_collect(solver_needs_opts.apply(lambda x : ";".join(x)   ,axis=1), delimiter=';')
solver_needs_opts

['Noval',
 'Public Relations (e.g. branding/marketing strategy, social and global media)',
 'Financial (e.g. improving accounting practices, pitching to investors)',
 'Business model (e.g. product-market fit, strategy & development)',
 'Human Capital (e.g. sourcing talent, board development, etc.)',
 'Product / Service Distribution (e.g. expanding client base)',
 'Technology (e.g. software or hardware, web development/design, data analysis, etc.)',
 'Legal or Regulatory Matters',
 'Other',
 'Monitoring & Evaluation (e.g. collecting/using data, measuring impact)',
 'Human Capital (i.e. sourcing talent, board development, etc.)']

In [13]:
# Check if all the solver need options are in partner preferences
set(solver_needs_opts).difference(set(partner_prefs_opts))

{'Human Capital (i.e. sourcing talent, board development, etc.)', 'Other'}

In [14]:
# Check the opposite of the above
set(partner_prefs_opts).difference(set(solver_needs_opts))

{'Other (explain below)'}

In [15]:
# Correcting Human Captial in partner data
wrong_str1 = 'Human Capital (e.g. sourcing talent, board development, etc.)'
wrong_str2 = 'Other (explain below)'
correct_str1 = 'Other'
correct_str2 = 'Human Capital (i.e. sourcing talent, board development, etc.)'
for pref in prefs_columns:
    partner_data_updated[pref] = partner_data_updated[pref].apply(lambda x: x.replace(wrong_str1, correct_str1) if x == wrong_str1 else x )
    partner_data_updated[pref] = partner_data_updated[pref].apply(lambda x: x.replace(wrong_str2, correct_str2) if x == wrong_str2 else x)

    

In [16]:
# Correcting Capital in solver data
wrong_str1 = 'Human Capital (e.g. sourcing talent, board development, etc.)'
correct_str2 = 'Human Capital (i.e. sourcing talent, board development, etc.)'
solver_data_updated = solver_data.copy()
for pref in needs_columns:
    solver_data_updated[pref] = solver_data_updated[pref].apply(lambda x: x.replace(wrong_str1, correct_str1) if x == wrong_str1 else x )
    


In [17]:
# Showing the partnership options after correcting for human capital
needs_columns = [x  for x in solver_data_updated.columns if needs_name in x]
solver_needs_opts = solver_data_updated[needs_columns]
solver_needs_opts = split_collect(solver_needs_opts.apply(lambda x : ";".join(x)   ,axis=1), delimiter=';')
solver_needs_opts


['Noval',
 'Other',
 'Public Relations (e.g. branding/marketing strategy, social and global media)',
 'Financial (e.g. improving accounting practices, pitching to investors)',
 'Business model (e.g. product-market fit, strategy & development)',
 'Product / Service Distribution (e.g. expanding client base)',
 'Technology (e.g. software or hardware, web development/design, data analysis, etc.)',
 'Legal or Regulatory Matters',
 'Monitoring & Evaluation (e.g. collecting/using data, measuring impact)',
 'Human Capital (i.e. sourcing talent, board development, etc.)']

## Stage Match

In [18]:
partner_data_updated['Solution Preference: Organization Stage'] = partner_data_updated['Solution Preference: Organization Stage'].apply(lambda x : x.replace(';',',' if ';' in x else x))
partner_stage = partner_data_updated['Solution Preference: Organization Stage']
partner_stage_opts = split_collect(partner_stage)


In [19]:
solver_stage_opts = split_collect(solver_data_updated['Stage'])


In [20]:
set(solver_stage_opts).difference(set(partner_stage_opts))

set()

No difference! means all the options are the same! 

### Add seprated stage columns to partner data sheet


In [21]:
partner_stage_cols = expand_col(partner_stage, col_name='Stage')
partner_data_updated = pd.concat([partner_data_updated, partner_stage_cols], axis=1)
partner_data_updated = partner_data_updated.drop(columns=['Solution Preference: Organization Stage'])


## Tech Match

In [22]:
partner_tech_opts = split_collect(partner_data_updated['Technology Expertise'])
tech_name = 'Tech'
tech_cols = [x for x in solver_data_updated.columns if  tech_name in x]
tech_cols.append('Org')
solver_tech_cols = solver_data_updated[tech_cols]
solver_tech_opts = split_collect(solver_tech_cols.apply(lambda x : ",".join(x), axis=1))
set(partner_tech_opts).difference(solver_tech_opts)

{'Imaging and Sensor Technology', 'Virtual Reality / Augmented Reality'}

In [2]:
partner_tech_cols = expand_col(partner_data_updated['Technology Expertise'], col_name='tech')
partner_data_updated = pd.concat([partner_data_updated, partner_tech_cols], axis=1)
partner_data_updated = partner_data_updated.drop(columns=['Technology Expertise'])

NameError: name 'expand_col' is not defined

# All matching sheets calculated here

##  Challenge Matching sheet


In [24]:
partner_data_updated = partner_data_updated.replace('None', 'Noval' ,regex=True)
solver_data_updated = solver_data_updated.replace('None', 'Noval', regex=True)
partner_data_updated = partner_data_updated.replace(np.nan, "Noval")
solver_data_updated  = solver_data_updated.replace(np.nan, "Noval")
partner_data_updated.columns

Index(['Unnamed: 0', 'Organization ID', 'Org', 'First Name', 'Last Name',
       'Type', 'Funding Preference 1', 'Funding Preference 2',
       'Funding Preference 3', 'Funding Preference 4',
       'Partnership Preference: Non-Financial 1',
       'Partnership Preference: Non-Financial 2',
       'Partnership Preference: Non-Financial 3',
       'Partnership Preference: Non-Financial 4',
       'Partnership Preference: Non-Financial 5',
       'Partnership Preference: Non-Financial 6',
       'Partnership Preference: Non-Financial 7', 'Challenge_1', 'Challenge_2',
       'Challenge_3', 'Challenge_4', 'Challenge_5', 'Challenge_6',
       'Challenge_7', 'Challenge_8', 'Challenge_9', 'Challenge_10',
       'Challenge_11', 'Challenge_12', 'Challenge_13', 'Challenge_14', 'geo_1',
       'geo_2', 'geo_3', 'geo_4', 'geo_5', 'geo_6', 'geo_7', 'geo_8',
       'Stage_1', 'Stage_2', 'Stage_3', 'Stage_4', 'Stage_5', 'tech_1',
       'tech_2', 'tech_3', 'tech_4', 'tech_5', 'tech_6', 'tech_7', 'tec

In [25]:
chname = 'Challenge'
challenge_cols = [x for x in partner_data_updated.columns if chname in x ]
partner_challenge_cols = partner_data_updated[challenge_cols]

In [26]:

solver_challenge_cols = solver_data_updated[['Org', 'Challenge']]
partner_challenge_cols['Org'] = partner_data_updated['Org']
challenge_match = match_single_to_multi(solver_challenge_cols, partner_challenge_cols, single_match_on='Challenge')

if 'Noval' in challenge_match.columns: 
    challenge_match = challenge_match.drop(columns=['Noval'])
challenge_match.shape

/home/pawan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(78, 43)

In [27]:
# Opening up single to multi function 

# single_df = solver_challenge_cols
# multi_df = partner_challenge_cols
# single_match_on = 'Challenge'

# melted_df = pd.melt(multi_df,id_vars='Org')
# melted_df = melted_df.drop(columns='variable')
# melted_df = melted_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# single_df = single_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# matched_df = pd.merge(melted_df, single_df, how='outer', left_on='value', right_on=single_match_on)
# matched_df = matched_df.replace(np.nan, 'Noval')
# matched_df['value'] = matched_df['value'].apply(lambda x : 0 if x == None else 1)
# pivot_table = pd.pivot_table(matched_df, index='Org_x', columns=['Org_y'], values='value', dropna=False,  aggfunc=np.sum)
# pivot_table.shape

#### Trouble shooting no match 

In [28]:
# unshared_challenges_partners = set(partner_challenge_cols['Org'].to_list()).difference(set(pivot_table.index.to_list()))
# challenges_missing = set(partner_challenge_opts).difference(set(solver_challenge_opts))
# missing_indx = partner_challenge_cols[partner_challenge_cols.isin(challenges_missing)].any(1).to_numpy().nonzero()
# possible_missing = partner_challenge_cols['Org'].loc[missing_indx].to_list()


This basically means that there are only 35 rows have matches, otherwise no matches

## Geo Matching sheet

In [29]:
geo_cols = [x for x in partner_data_updated.columns if 'geo' in x]
geo_cols.append('Org')
partner_geo_cols = partner_data_updated[geo_cols]

solver_geo_cols = solver_data_updated[['Org', 'Geo 1', 'Geo 2', 'Geo 3']]
geo_match = match_multi(partner_geo_cols,solver_geo_cols)
if 'Noval' in geo_match.columns: 
    geo_match = geo_match.drop(columns=['Noval'])
geo_match.shape

(78, 43)

## Needs Matching sheet

In [30]:
needs_name ='Key Need'
pref_name = 'Partnership Preference'
prefs_columns = [x  for x in partner_data_updated.columns if pref_name in x]
needs_columns = [x  for x in solver_data_updated.columns if needs_name in x]
prefs_columns.append('Org')
needs_columns.append('Org')
partner_prefs_cols = partner_data_updated[prefs_columns]
solver_needs_cols = solver_data_updated[needs_columns]
needs_match = match_multi(partner_prefs_cols, solver_needs_cols)
if 'Noval' in needs_match.columns: 
    needs_match = needs_match.drop(columns=['Noval'])
needs_match.shape

(78, 43)

## Stage Matching sheet

In [31]:
stage_name = 'Stage'
stage_columns = [x  for x in partner_data_updated.columns if stage_name in x]
stage_columns.append('Org')
partner_stage_cols = partner_data_updated[stage_columns]
solver_stage_cols = solver_data_updated[['Stage', 'Org']]
stage_match = match_single_to_multi(solver_stage_cols, partner_stage_cols, single_match_on='Stage')

if 'Noval' in stage_match.columns: 
    stage_match = stage_match.drop(columns=['Noval'])
stage_match.shape

(78, 43)

In [32]:
set(stage_match.columns.to_list()).difference(set(needs_match.columns.to_list()))

set()

## Tech Matching Sheet

In [33]:
tech_name = 'tech'
tech_cols = [x for x in partner_data_updated.columns if  tech_name in x]
tech_cols.append('Org')
partner_tech_cols = partner_data_updated[tech_cols]

In [34]:
partner_tech_cols['Org'] = partner_data_updated['Org']
tech_match = match_multi(partner_tech_cols, solver_tech_cols)
if 'Noval' in tech_match.columns:
    tech_match = tech_match.drop(columns=['Noval'])
if 'Noval' in tech_match.index:
    tech_match = tech_match.drop(index='Noval', axis=0) 
tech_match.shape

/home/pawan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(78, 43)

## Summing all sheets 


In [35]:
# tech_matched = tech_matched.fillna(0)
needs_match = needs_match.fillna(0)
geo_match = geo_match.fillna(0)
stage_match= stage_match.fillna(0)
challenge_match = challenge_match.fillna(0)

partner_solver_weights = pd.read_excel('input_data/2020_input_data.xlsx', sheet_name= 'Partner Solver Weights')
geo_weights_pivot = pd.pivot(partner_solver_weights[['Org_y', 'Org_x', 'geo_weights']], columns='Org_x', index='Org_y' )
needs_weights_pivot = pd.pivot(partner_solver_weights[['Org_y', 'Org_x', 'needs_weights']], columns='Org_x', index='Org_y' )
challenge_weights_pivot = pd.pivot(partner_solver_weights[['Org_y', 'Org_x', 'challenge_weights']], columns='Org_x', index='Org_y' )
stage_weights_pivot = pd.pivot(partner_solver_weights[['Org_y', 'Org_x', 'stage_weights']], columns='Org_x', index='Org_y' )

challenge_term = 10*pd.DataFrame(challenge_weights_pivot.values*challenge_match.astype(float).values, columns=challenge_weights_pivot.columns, index=challenge_weights_pivot.index)['challenge_weights']
stage_term = pd.DataFrame(stage_weights_pivot.values*stage_match.astype(float).values, columns=stage_weights_pivot.columns, index=stage_weights_pivot.index)
geo_term = pd.DataFrame(geo_weights_pivot.values*geo_match.astype(float).values, columns=geo_weights_pivot.columns, index=geo_weights_pivot.index)['geo_weights']
geo_stage_term = 100*pd.DataFrame(geo_term.values*stage_term.values, columns=stage_term.columns, index=stage_term.index)['stage_weights']
needs_term =  pd.DataFrame(needs_weights_pivot.values*needs_match.astype(float).values, columns=needs_weights_pivot.columns, index=needs_weights_pivot.index)['needs_weights']

total_score = challenge_term  + geo_stage_term + needs_term
# partner_solver_weights['Org_x'].value_counts().index

In [36]:
partner_data_updated = partner_data_updated.replace('None', 'Noval' ,regex=True)
solver_data_updated = solver_data_updated.replace('None', 'Noval', regex=True)
partner_data_updated = partner_data_updated.replace(np.nan, "Noval")
solver_data_updated  = solver_data_updated.replace(np.nan, "Noval")

file_name = '2020_data.xlsx'
with pd.ExcelWriter(file_name) as writer: 
    partner_data_updated.to_excel(writer, sheet_name='Partner Data', index=False)
    solver_data_updated.to_excel(writer, sheet_name='Solver Team Data', index= False)
writer.close()

## Check sort values for the output graphs


In [41]:
uploaded_df_total_score = pd.read_excel("input_data/total_score_from_upload.xlsx", sheet_name="Sheet1")



In [42]:
value = 'EA Ecoversity'
solver_scores = uploaded_df_total_score[['Org_y', value]]
uploaded_df_total_score.sort_values(value,  ascending=False)[:5]
solver_scores.sort_values(by=[ value], ascending=False)

,Org_y,EA Ecoversity
16,Comcast NBCUniversal,13.0
58,Someone Else's Child Foundation,12.0
6,BMW Foundation Herbert Quandt,11.0
40,Llamasoft,6.0
44,Merian Ventures,6.0
...,...,...
46,Morgridge Family Foundation,1.0
29,"Global Fund to fight Aids, Tuberculosis and Ma...",0.0
35,KSF Impact,0.0
38,Leap Ventures,0.0


In [43]:
uploaded_df_total_score.sort_values(value,  ascending=False)[:5]


,Org_y,AHSA Platform,Amplify Her Voice,Asia Initiatives Learning Cascades,Bambara Milk,Beewise,Bioforge Neonatal Incubator,Biometricsfor vaccine delivery,D2,Democratizing Ultrasound Africa,...,Someone Somewhere,Symbrosia,TamoJunto.org.br,Thaki,The Last Mile,Ubenwa,Universally Friendly Obturator,Whole Surplus,Yiya AirScience,eggXYt
16,Comcast NBCUniversal,4,3,6,1,2,2,3,12,2,...,13,4,13,5,14,2,6,2,5,2
58,Someone Else's Child Foundation,6,14,16,2,2,2,4,12,3,...,14,4,14,14,13,2,6,2,16,3
6,BMW Foundation Herbert Quandt,2,2,4,1,2,0,3,10,3,...,14,1,13,4,13,1,2,2,2,1
40,Llamasoft,4,12,17,12,15,11,2,3,13,...,5,12,3,16,4,13,13,14,14,12
44,Merian Ventures,6,2,6,11,16,1,2,12,2,...,13,13,12,4,14,3,3,16,5,11


In [49]:

solver_scores[solver_scores['Org_y'] == 'Nuvo']

49    3.0
Name: EA Ecoversity, dtype: float64

In [52]:
uploaded_df_total_score.sort_values(value,  ascending=False)[:1]['D2']


16    12
Name: D2, dtype: int64

In [61]:
solver_options = solver_data_updated['Org']
solver_options = solver_options.to_frame(name='Solvers')
matches = ['None' for x in range(0, solver_options.shape[0])]
solver_options['matches'] = matches
solver_options

,Solvers,matches
0,AHSA Platform,None
1,Biometricsfor vaccine delivery,None
2,Jute-based biodegradable PPE,None
3,MapSights,None
4,Mosquito-borne disease prevention,None
5,PENSA *660#,None
6,PODD,None
7,EA Ecoversity,None
8,Food from Fire,None
9,Indigenous DC,None


In [80]:
add_val = "D2"
row_indx = solver_options[solver_options['Solvers'] == 'Thaki'].index[0]
solver_options.iloc[row_indx, 1]

'None'

In [81]:


def update_colval(df,add_term, row_val, col, col2): 
    col_indx = df[df[col] == row_val].index.values[0]
    cell_val = df.at[col_indx, col2]
    cell_val = cell_val.split(',')
    if add_term in cell_val: 
        return  1 
    else: 
        cell_val.append(add_term)
        count_value = len(cell_val)-1 
        cell_val = ",".join(cell_val)
        df.at[col_indx, col2] = cell_val
        df.at[col_indx, 'Count'] =  count_value 
        return df, count_value

In [85]:
update_colval(solver_options,add_val, "Thaki", "Solvers", "matches")    

(                                 Solvers            matches  Count
 0                          AHSA Platform               None    NaN
 1         Biometricsfor vaccine delivery               None    NaN
 2           Jute-based biodegradable PPE               None    NaN
 3                              MapSights               None    NaN
 4      Mosquito-borne disease prevention               None    NaN
 5                            PENSA *660#               None    NaN
 6                                   PODD               None    NaN
 7                          EA Ecoversity               None    NaN
 8                         Food from Fire               None    NaN
 9                          Indigenous DC               None    NaN
 10                         Indigikitchen               None    NaN
 11  International Wakashan AI-Consortium               None    NaN
 12              Protect Medicinal Plants               None    NaN
 13                             ShockTalk       

In [93]:
solver_opts = pd.read_excel('input_data/solver_options.xlsx')
solver_opts[solver_opts['Solvers'] == 'AHSA Platform']['matches'][0].split(',')

['None',
 'The Kamath Family Foundation',
 'Usizo Advisory Solutions',
 'Kevin Przybocki']